In [1]:
import lammps

from pathlib import Path
import pandas as pd

## Defining constants

In [2]:
ENERGY_EVAL_INP = '''
units real
atom_style full

dimension 3
boundary p p p

pair_style lj/cut/coul/cut 20.0 20.0
pair_modify mix arithmetic

bond_style hybrid harmonic
angle_style hybrid harmonic
dihedral_style hybrid fourier
improper_style cvff
special_bonds lj 0.0 0.0 0.5 coul 0.0 0.0 0.8333

read_data $INP_FILE
thermo_style yaml ebond eangle edihed eimp epair evdwl ecoul elong etail pe ke etotal
run 0
'''

E_ALL = {
    'Pairwise (kcal/mol)'       : 'epair',
    'Bond (kcal/mol)'           : 'ebond',
    'Angle (kcal/mol)'          : 'eangle',
    'Dihedral (kcal/mol)'       : 'edihed',
    'Impropers (kcal/mol)'      : 'eimp',
    'vdW pairwise (kcal/mol)'   : 'evdwl',
    'Coulomb (kcal/mol)'        : 'ecoul',
    'Ranged k-space (kcal/mol)' : 'elong',
    'vdW ranged (kcal/mol)'     : 'etail',
    'Potential (kcal/mol)'      : 'pe',
    'Kinetic (kcal/mol)'        : 'ke',
    'Total (kcal/mol)'          : 'etotal'
}

E_TO_REPORT = {
    'Nonbonded (kcal/mol)' : 'epair',
    'Bond (kcal/mol)'      : 'ebond',
    'Angle (kcal/mol)'     : 'eangle',
    'Torsion (kcal/mol)'   : 'edihed',
    'Impropers (kcal/mol)' : 'eimp',
    'Potential (kcal/mol)' : 'pe',
    'Kinetic (kcal/mol)'   : 'ke',
    # 'Total (kcal/mol)'     : 'etotal'
}

CELL_KW = (
    'cella',
    'cellb',
    'cellc',
    'cellalpha',
    'cellbeta',
    'cellgamma',
)

In [3]:
lammps_dir = Path('LAMMPS')

records = {}
cell_sizes = {}
for subdir in lammps_dir.iterdir():
    if subdir.is_dir():
        chemistry = subdir.name
        for lammps_file in subdir.iterdir():
            mol_name = lammps_file.stem
            with lammps.lammps() as lmp: # need to create new lammps() object instance for each run
                lmp.commands_string( ENERGY_EVAL_INP.replace('$INP_FILE', str(lammps_file)) )
                rep_energies = {
                    label : lmp.get_thermo(prop_tag)
                        # for label, prop_tag in E_TO_REPORT.items()
                        for label, prop_tag in E_ALL.items()
                }

                cell_params = {
                    cp : lmp.get_thermo(cp)
                        for cp in CELL_KW
                }

            records[(chemistry, mol_name)] = rep_energies
            cell_sizes[(chemistry, mol_name)] = cell_params

hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


LAMMPS (28 Mar 2023 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (-14.49 -2.589 -2.2490001) to (85.51 97.411 97.751) with tilt (0 0 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  540 atoms
  scanning bonds ...
  3 = max bonds/atom
  scanning angles ...
  6 = max angles/atom
  scanning dihedrals ...
  23 = max dihedrals/atom
  scanning impropers ...
  2 = max impropers/atom
  reading bonds ...
  549 bonds
  reading angles ...
  972 angles
  reading dihedrals ...
  1848 dihedrals
  reading impropers ...
  267 impropers
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0.5     
  special bond factors coul:  0        0        0.8333  
     4 = max # of 1-2 neighbors
     7 = max # of 1-3 neighbors
    13 = max # of 1-4 neighbors
    16 = max # of special neighbors
  special bonds CPU = 0.001 seconds
  read_data CPU = 0.0

In [4]:
lmp_df = pd.DataFrame.from_dict(records, 'index')
lmp_df.index.names  = ['Chemistry', 'Molecule'] # ensure index labels are labelled consistently
lmp_df.sort_values('Molecule', inplace=True)
lmp_df.to_csv(lammps_dir/'LAMMPS_PEs.csv')

In [5]:
cell_sizes

{('NIPU', 'NIPU_11'): {'cella': 100.0,
  'cellb': 99.99999999699999,
  'cellc': 100.00000000200001,
  'cellalpha': 90.0,
  'cellbeta': 90.0,
  'cellgamma': 90.0},
 ('NIPU', 'NIPU_2'): {'cella': 99.999999997,
  'cellb': 100.000000004,
  'cellc': 100.0,
  'cellalpha': 90.0,
  'cellbeta': 90.0,
  'cellgamma': 90.0},
 ('urethane', 'urethane_44'): {'cella': 100.0,
  'cellb': 100.0,
  'cellc': 100.000000004,
  'cellalpha': 90.0,
  'cellbeta': 90.0,
  'cellgamma': 90.0},
 ('urethane', 'urethane_51'): {'cella': 100.0,
  'cellb': 100.0,
  'cellc': 100.0,
  'cellalpha': 90.0,
  'cellbeta': 90.0,
  'cellgamma': 90.0},
 ('urethane', 'urethane_24'): {'cella': 100.0,
  'cellb': 100.0,
  'cellc': 100.0,
  'cellalpha': 90.0,
  'cellbeta': 90.0,
  'cellgamma': 90.0},
 ('urethane', 'urethane_8'): {'cella': 100.0,
  'cellb': 100.0,
  'cellc': 100.0,
  'cellalpha': 90.0,
  'cellbeta': 90.0,
  'cellgamma': 90.0},
 ('urethane', 'urethane_17'): {'cella': 100.0,
  'cellb': 100.0,
  'cellc': 99.999999998,
  'c